In [89]:
### Run this cell before continuing.
library(tidyverse)
library(tidymodels)
library(tidyclust)
library(forcats)
library(repr)
library(lubridate)
library(dplyr)
options(repr.matrix.max.rows = 6)
#source("cleanup.R")

Read in the players.csv data using read_csv() and assign it to an object called players. The data is located within the data/ folder.


In [90]:
players <- read_csv("Data/players.csv")
players

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,FALSE,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db299bd4fedb06a46ad5bb,0.0,Dylan,Prefer not to say,17
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17
Pro,TRUE,d9473710057f7d42f36570f0be83817a4eea614029ff90cf50d8889cdd729d11,0.2,Ahmed,Other,NA


Read in the sessions.csv data using read_csv() and assign it to an object called sessions. The data is located within the data/ folder.

In [91]:
sessions <- read_csv("Data/sessions.csv")
sessions

Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


hashedEmail,start_time,end_time,original_start_time,original_end_time
<chr>,<chr>,<chr>,<dbl>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,30/06/2024 18:12,30/06/2024 18:24,1.71977e+12,1.71977e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,17/06/2024 23:33,17/06/2024 23:46,1.71867e+12,1.71867e+12
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,25/07/2024 17:34,25/07/2024 17:57,1.72193e+12,1.72193e+12
⋮,⋮,⋮,⋮,⋮
fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33cbb5e894a3867ca44d,28/07/2024 15:36,28/07/2024 15:57,1.72218e+12,1.72218e+12
fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33cbb5e894a3867ca44d,25/07/2024 06:15,25/07/2024 06:22,1.72189e+12,1.72189e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,20/05/2024 02:26,20/05/2024 02:45,1.71617e+12,1.71617e+12


Merge the datasets on hashedEmail using the inner_join() function. This is useful for combining player characteristics with their session data, which is essential to analyze how player behavior (from sessions.csv) relates to subscription status (from players.csv).

In [92]:
merged_data <- inner_join(players, sessions, by = "hashedEmail")
merged_data

experience,subscribe,hashedEmail,played_hours,name,gender,Age,start_time,end_time,original_start_time,original_end_time
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,08/08/2024 00:21,08/08/2024 01:35,1.72308e+12,1.72308e+12
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,09/09/2024 22:30,09/09/2024 22:37,1.72592e+12,1.72592e+12
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,08/08/2024 02:41,08/08/2024 03:25,1.72308e+12,1.72309e+12
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17,23/08/2024 21:39,23/08/2024 21:53,1.72445e+12,1.72445e+12
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17,08/09/2024 19:40,08/09/2024 19:45,1.72582e+12,1.72582e+12
Pro,TRUE,d9473710057f7d42f36570f0be83817a4eea614029ff90cf50d8889cdd729d11,0.2,Ahmed,Other,NA,08/05/2024 19:29,08/05/2024 19:44,1.71520e+12,1.71520e+12


View your new data frame, by using the glimpse() function.

In [93]:
glimpse(merged_data)

Rows: 1,535
Columns: 11
$ experience          <chr> "Pro", "Pro", "Pro", "Pro", "Pro", "Pro", "Pro", "…
$ subscribe           <lgl> TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TR…
$ hashedEmail         <chr> "f6daba428a5e19a3d47574858c13550499be23603422e6a0e…
$ played_hours        <dbl> 30.3, 30.3, 30.3, 30.3, 30.3, 30.3, 30.3, 30.3, 30…
$ name                <chr> "Morgan", "Morgan", "Morgan", "Morgan", "Morgan", …
$ gender              <chr> "Male", "Male", "Male", "Male", "Male", "Male", "M…
$ Age                 <dbl> 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,…
$ start_time          <chr> "08/08/2024 00:21", "09/09/2024 22:30", "08/08/202…
$ end_time            <chr> "08/08/2024 01:35", "09/09/2024 22:37", "08/08/202…
$ original_start_time <dbl> 1.72308e+12, 1.72592e+12, 1.72308e+12, 1.72598e+12…
$ original_end_time   <dbl> 1.72308e+12, 1.72592e+12, 1.72309e+12, 1.72598e+12…


In [96]:
merged_data_1 <-  merged_data |>
  mutate(
    start_time = dmy_hm(start_time),
    end_time = dmy_hm(end_time),
    session_length_min = as.numeric(difftime(end_time, start_time, units = "mins"))   
  )
merged_data_1

experience,subscribe,hashedEmail,played_hours,name,gender,Age,start_time,end_time,original_start_time,original_end_time,session_length_min
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dttm>,<dttm>,<dbl>,<dbl>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,2024-08-08 00:21:00,2024-08-08 01:35:00,1.72308e+12,1.72308e+12,74
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,2024-09-09 22:30:00,2024-09-09 22:37:00,1.72592e+12,1.72592e+12,7
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,2024-08-08 02:41:00,2024-08-08 03:25:00,1.72308e+12,1.72309e+12,44
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17,2024-08-23 21:39:00,2024-08-23 21:53:00,1.72445e+12,1.72445e+12,14
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17,2024-09-08 19:40:00,2024-09-08 19:45:00,1.72582e+12,1.72582e+12,5
Pro,TRUE,d9473710057f7d42f36570f0be83817a4eea614029ff90cf50d8889cdd729d11,0.2,Ahmed,Other,NA,2024-05-08 19:29:00,2024-05-08 19:44:00,1.71520e+12,1.71520e+12,15


In [ ]:
text

In [99]:
# Aggregate session info per player
session_summary <- merged_data_1 |>
  group_by(hashedEmail) |>
  summarize(avg_session_length_min = mean(session_length_min, na.rm = TRUE), 
  num_sessions = n()
  )
session_summary


# Merge session summary back to player-level data
player_data <- inner_join(players, session_summary, by = "hashedEmail")
player_data

hashedEmail,avg_session_length_min,num_sessions
<chr>,<dbl>,<int>
0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc93355ccc95b45423367832,53,2
060aca80f8cfbf1c91553a72f4d5ec8034764b05ab59fe8e1cf0eee9a7b67967,30,1
0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce0290fa0437ce0b97f387,11,1
⋮,⋮,⋮
fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33cbb5e894a3867ca44d,15.46774,310
fe218a05c6c3fc6326f4f151e8cb75a2a9fa29e22b110d4c311fb58fb211f471,9.00000,1
fef4e1bed8c3f6dcd7bcd39ab21bd402386155b2ff8c8e53683e1d2793bf1ed1,72.00000,1


experience,subscribe,hashedEmail,played_hours,name,gender,Age,avg_session_length_min,num_sessions
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<int>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,74.77778,27
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17,85.00000,3
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17,5.00000,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,FALSE,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db299bd4fedb06a46ad5bb,0.0,Dylan,Prefer not to say,17,5.00000,1
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17,29.83333,6
Pro,TRUE,d9473710057f7d42f36570f0be83817a4eea614029ff90cf50d8889cdd729d11,0.2,Ahmed,Other,NA,15.00000,1


text

In [101]:
model_data <- player_data |>
  select(subscribe, experience, played_hours, Age, gender, avg_session_length_min, num_sessions) |>
  drop_na()
model_data

subscribe,experience,played_hours,Age,gender,avg_session_length_min,num_sessions
<lgl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<int>
TRUE,Pro,30.3,9,Male,74.77778,27
TRUE,Veteran,3.8,17,Male,85.00000,3
FALSE,Veteran,0.0,17,Male,5.00000,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮
FALSE,Veteran,0.3,22,Male,21.00000,1
FALSE,Amateur,0.0,17,Prefer not to say,5.00000,1
FALSE,Amateur,2.3,17,Male,29.83333,6


text

In [102]:
model_data <- model_data |>
  mutate(subscribe = as.factor(subscribe))
model_data

subscribe,experience,played_hours,Age,gender,avg_session_length_min,num_sessions
<fct>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<int>
TRUE,Pro,30.3,9,Male,74.77778,27
TRUE,Veteran,3.8,17,Male,85.00000,3
FALSE,Veteran,0.0,17,Male,5.00000,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮
FALSE,Veteran,0.3,22,Male,21.00000,1
FALSE,Amateur,0.0,17,Prefer not to say,5.00000,1
FALSE,Amateur,2.3,17,Male,29.83333,6
